In [1]:
import unicodedata
import re
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
credits_df = pd.read_csv('../data/clean/clean_credits.csv')
keywords_df = pd.read_csv('../data/clean/clean_keywords.csv')
movies_metadata_df = pd.read_csv('../data/clean/clean_movies_metadata.csv')

In [3]:
df=(credits_df.merge(movies_metadata_df,on='id')).merge(keywords_df,on='id')

In [4]:
df.head(n=3)

,Unnamed: 0_x,cast,id,director,Unnamed: 0_y,belongs_to_collection,budget,genres,original_language,popularity,...,runtime,status,title,vote_average,vote_count,release_year,profit,weighted_rating,Unnamed: 0,keywords
0,0,"['Tom Hanks', 'Tim Allen', 'Don Rickles']",862,John Lasseter,0,1,0.078947,"['Animation', 'Comedy', 'Family']",en,21.946943,...,81.0,Released,Toy Story,7.7,5415.0,1995.0,0.904090,7.684684,0,"['new toy', 'boy next door', 'toy comes to lif..."
1,1,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",8844,Joe Johnston,1,0,0.171053,"['Adventure', 'Fantasy', 'Family']",en,17.015540,...,104.0,Released,Jumanji,6.9,2413.0,1995.0,0.520519,6.877012,1,"['recluse', 'board game', 'new home', 'giant i..."
2,2,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']",15602,Howard Deutch,2,1,NaN,"['Romance', 'Comedy']",en,11.712900,...,101.0,Released,Grumpier Old Men,6.5,92.0,1995.0,NaN,6.163983,2,"['fishing', 'best friend', 'duringcreditssting..."


In [5]:
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31641 entries, 0 to 31640
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cast                   30752 non-null  object 
 1   id                     31641 non-null  int64  
 2   director               31508 non-null  object 
 3   belongs_to_collection  31641 non-null  int64  
 4   budget                 7815 non-null   float64
 5   genres                 31641 non-null  object 
 6   original_language      31641 non-null  object 
 7   popularity             31641 non-null  float64
 8   production_companies   31641 non-null  object 
 9   production_countries   31641 non-null  object 
 10  release_date           31641 non-null  object 
 11  revenue                6838 non-null   float64
 12  runtime                31641 non-null  float64
 13  status                 31641 non-null  object 
 14  title                  31641 non-null  object 
 15  vo

In [6]:
training_df = df

In [7]:
training_df['status'].value_counts()

Released           31470
Rumored              101
Post Production       51
Planned               10
In Production          8
Canceled               1
Name: status, dtype: int64

In [8]:
training_df=training_df[training_df['status']=='Released']
training_df.drop('status', axis=1, inplace=True)

/home/taquan/Desktop/DHBK/20211/IntroToDS/code/venv/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
training_df=training_df[['title','cast','director','genres','keywords']]

In [10]:
training_df.head(n=3)

,title,cast,director,genres,keywords
0,Toy Story,"['Tom Hanks', 'Tim Allen', 'Don Rickles']",John Lasseter,"['Animation', 'Comedy', 'Family']","['new toy', 'boy next door', 'toy comes to lif..."
1,Jumanji,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",Joe Johnston,"['Adventure', 'Fantasy', 'Family']","['recluse', 'board game', 'new home', 'giant i..."
2,Grumpier Old Men,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']",Howard Deutch,"['Romance', 'Comedy']","['fishing', 'best friend', 'duringcreditssting..."


In [11]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31470 entries, 0 to 31640
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     31470 non-null  object
 1   cast      30594 non-null  object
 2   director  31338 non-null  object
 3   genres    31470 non-null  object
 4   keywords  31470 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [12]:
training_df.isnull().sum()

title         0
cast        876
director    132
genres        0
keywords      0
dtype: int64

In [13]:
training_df.dropna(inplace=True)

In [14]:
training_df.isnull().sum()

title       0
cast        0
director    0
genres      0
keywords    0
dtype: int64

In [15]:
training_df[training_df['title'].str.find('Wife') != -1]

,title,cast,director,genres,keywords
712,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
713,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
714,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
715,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
716,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
717,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
718,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
719,Wife,"['Mieko Takamine', 'Ken Uehara', 'Rentarô Miku...",Mikio Naruse,['Drama'],"['based on novel', 'wife', 'marriage', 'unhapp..."
841,The Rich Man's Wife,"['Clive Owen', 'Halle Berry', 'Peter Greene']",Amy Holden Jones,"['Drama', 'Thriller']","['surprise', 'rich', 'stranger', 'fantasy', 'm..."
986,"The Cook, the Thief, His Wife & Her Lover","['Richard Bohringer', 'Michael Gambon', 'Helen...",Peter Greenaway,"['Crime', 'Drama']","['cook', 'cooking', 'restaurant', 'crime boss']"


In [16]:
training_df.drop_duplicates("title", inplace=True)
training_df = training_df.reset_index()

In [17]:
def clean_data(value):
    value =  unicodedata.normalize('NFD', value).encode('ascii', 'ignore').decode('ascii')
    x= str.lower(re.sub(r"[^a-zA-Z0-9]+", '_', value.strip().replace(" ","_")))
    if(x!='_'):
        return x 
    else:
        return " "

In [18]:
training_df['director'] = training_df['director'].apply(lambda x: clean_data(x))
training_df['cast'] = training_df['cast'].apply(lambda x : [clean_data(i) for i in literal_eval(x)])
training_df['genres'] = training_df['genres'].apply(lambda x : [clean_data(i) for i in literal_eval(x)])
training_df['keywords'] = training_df['keywords'].apply(lambda x : [clean_data(i) for i in literal_eval(x)])

In [19]:
training_df.head(n=3)

,index,title,cast,director,genres,keywords
0,0,Toy Story,"[tom_hanks, tim_allen, don_rickles]",john_lasseter,"[animation, comedy, family]","[new_toy, boy_next_door, toy_comes_to_life, to..."
1,1,Jumanji,"[robin_williams, jonathan_hyde, kirsten_dunst]",joe_johnston,"[adventure, fantasy, family]","[recluse, board_game, new_home, giant_insect, ..."
2,2,Grumpier Old Men,"[walter_matthau, jack_lemmon, ann_margret]",howard_deutch,"[romance, comedy]","[fishing, best_friend, duringcreditsstinger, o..."


In [20]:
training_df['bow']=training_df['cast']+training_df['keywords'] + training_df['genres']
training_df['bow']=training_df['bow'].apply(lambda x: ' '.join(x))+" "+training_df['director']

In [21]:
training_df["bow"].head(n=3)

0    tom_hanks tim_allen don_rickles new_toy boy_ne...
1    robin_williams jonathan_hyde kirsten_dunst rec...
2    walter_matthau jack_lemmon ann_margret fishing...
Name: bow, dtype: object

In [22]:
vectorizer = CountVectorizer(stop_words='english')
train_array = vectorizer.fit_transform(training_df['bow'])

In [23]:
cosine_sim = cosine_similarity(train_array, train_array)

In [24]:
training_df = training_df.reset_index()
indices = pd.Series(training_df.index, index=training_df['title'])
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
House of Horrors               28134
Shadow of the Blair Witch      28135
The Burkittsville 7            28136
Subdue                         28137
Century of Birthing            28138
Length: 28139, dtype: int64

In [25]:
def get_recommendations(title, cosine_sim= cosine_sim):
    if training_df[training_df['title'].str.find(title) != -1].shape[0] == 0:
        return "This film has not been in our database"
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return training_df['title'].iloc[movie_indices]

In [28]:
get_recommendations('Toy Story')

2578                                          Toy Story 2
18528                          Toy Story That Time Forgot
16312                                Toy Story of Terror!
19460                                               Anina
12248                                         Toy Story 3
18526                                     Partysaurus Rex
23920                                The Bear That Wasn't
27557                        Last Year's Snow Was Falling
9503                            The Ugly Duckling and Me!
4820                                   The Santa Clause 2
9142                                        Monster House
13572                                              Cars 2
4785          The Looney, Looney, Looney Bugs Bunny Movie
27836                                      Devil May Hare
10975          Kung Fu Panda: Secrets of the Furious Five
18350                         Scooby-Doo! Abracadabra-Doo
9175                                             Barnyard
1218          